In [27]:
# Dependencies and Setup
from datetime import datetime
import pandas as pd
import numpy as np
import os
import requests
import json
from pprint import pprint
from dotenv import load_dotenv

In [28]:
# Day of the week
today = datetime.today()
weekday = today.isoweekday()

days_week_dict = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday",
                 5: "Friday", 6: "Saturday", 7: "Sunday"}
day_of_the_week = days_week_dict.get(weekday)
print(day_of_the_week)

Monday


In [29]:
# Creates directory for current day of the week, raises an error if already exists.
parent_dir = 'json_files'
output_dir = f'{day_of_the_week}_Categories'
path = os.path.join(parent_dir, output_dir)

try:
    os.mkdir(path)
except:
    print('Folder already exists.')

Folder already exists.


In [30]:
# API Call
load_dotenv()
API_KEY = os.getenv("Yelp_API") # put your key here if you don't have a .env file in your main project folder.
headers = {'Authorization': 'Bearer %s' % API_KEY}
base_url = "https://api.yelp.com/v3/businesses/search"

In [31]:
stadium_data_df = pd.read_csv('Resources/stadium_coordinates.csv', dtype={
    "NFL Team": "string",
    "City": "string",
    "State": "string",
    "Stadium Name": "string",
    "Latitude": np.float64,
    "Longitude": np.float64,
    "Zip Code": "string"
})
stadium_data_df = stadium_data_df.dropna()
stadium_data_df.head()

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202


In [32]:
# Make a list of coordinate tuples (lat, lng, city) for each team
stadium_zip = zip(stadium_data_df["Latitude"], stadium_data_df["Longitude"], stadium_data_df['NFL Team'])
stadium = list(stadium_zip)
print(stadium)

[(33.528, -112.26299999999999, 'Arizona Cardinals'), (33.755556, -84.4, 'Atlanta Falcons'), (39.278056, -76.622778, 'Baltimore Ravens'), (42.773999999999994, -78.78699999999999, 'Buffalo Bills'), (35.225833, -80.852778, 'Carolina Panthers'), (41.8623, -87.6167, 'Chicago Bears'), (39.095, -84.516, 'Cincinnati Bengals'), (41.506111, -81.699444, 'Cleveland Browns'), (32.747778000000004, -97.09277800000001, 'Dallas Cowboys'), (39.741, -105.0197, 'Denver Broncos'), (42.34, -83.045556, 'Detroit Lions'), (44.501389, -88.06222199999999, 'Green Bay Packers'), (29.684721999999997, -95.410833, 'Houston Texans'), (39.760056, -86.16380600000001, 'Indianapolis Colts'), (30.323889, -81.6375, 'Jacksonville Jaguars'), (39.048889, -94.483889, 'Kansas City Chiefs'), (36.090833, -115.18361100000001, 'Las Vegas Raiders'), (33.95345, -118.3392, 'Los Angeles Chargers'), (33.95345, -118.3392, 'Los Angeles Rams'), (25.958056, -80.238889, 'Miami Dolphins'), (44.974, -93.258, 'Minnesota Vikings'), (42.091, -71.2

In [33]:
# API Parameters
search_limit = 20 #limit to 20 business names per query
radius = 3000 #radius in meters
term = "restaurants"
search_params = {"term": term, "limit": search_limit, "radius": radius, "sort_by": "rating", 
                 "categories":("sportsbars","pubs","wine_bars", "cocktailbars")}

# lists to store the data retrieve from Yelp
business_name = []
business_state = []
business_price = []
business_rating = []
business_reviews = []
number_business = []
#print(stadium)

In [34]:
# Create Exception

class Exception_1(Exception):
    """ Raised if parsing json failes """
    pass

In [35]:
#Loop to dump data into json files based on (lat,lng)
count_f = 0


for latitude, longitude, team in stadium:
        
    search_params.update({"latitude": latitude, "longitude": longitude})
    response = requests.get(url = base_url, params = search_params, headers = headers).json()
    try:
        
        if response['total']== False:
            raise Exception_1
        elif response['total'] >= 1000:
            total = 1000
        else:
            total = response['total']
        
        for search_offset in range(0, total, 20):
        
            search_params.update({"offset": search_offset})
            response2 = requests.get(url = base_url, params = search_params, headers = headers).json()
            file_name = f'yelp_category_response_{team}_{search_offset}.json'
            output_path = os.path.join(path, file_name)
            
            with open(output_path,'w', encoding = 'utf-8') as f:
                json.dump(response2, f, ensure_ascii=False, indent =4)
            f.close()
            
    except Exception_1 as e:
        print("Error parsing json.\nPlease try again later.")
        print(e)
        
    print(f'{count_f}, {team} total: {total}')
    
    count_f += 1
        

0, Arizona Cardinals total: 11
1, Atlanta Falcons total: 57
2, Baltimore Ravens total: 55
3, Buffalo Bills total: 3
4, Carolina Panthers total: 69
5, Chicago Bears total: 127
6, Cincinnati Bengals total: 57
7, Cleveland Browns total: 39
8, Dallas Cowboys total: 25
9, Denver Broncos total: 58
10, Detroit Lions total: 65
11, Green Bay Packers total: 10
12, Houston Texans total: 3
13, Indianapolis Colts total: 35
14, Jacksonville Jaguars total: 12
15, Kansas City Chiefs total: 1
16, Las Vegas Raiders total: 79
17, Los Angeles Chargers total: 4
18, Los Angeles Rams total: 4
Error parsing json.
Please try again later.

19, Miami Dolphins total: 4
20, Minnesota Vikings total: 64
21, New England Patriots total: 3
22, New Orleans Saints total: 97
23, New York Giants total: 5
24, New York Jets total: 5
25, Philadelphia Eagles total: 23
26, Pittsburgh Steelers total: 38
27, San Francisco 49ers total: 11
28, Seattle Seahawks total: 154
29, Tampa Bay Buccaners total: 15
30, Tennessee Titans total: